In [33]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
import ast
import sys

from pyarrow.parquet import ParquetFile
import pyarrow as pa 

ukb_file='../data/ukb50790.tab'
sys.path.append("../../ukb-dementia-shap/")

sys.path.append("../Pain/code/")


from logic.data_processing.data_setup import *
from logic.data_processing.data_import import dataload
from logic.data_processing.data_processing import data_proc_main




dp=data_proc_main()
ds=data_setup()



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
[f for f in os.listdir(dp.path) if 'feath' in f]

['0.feather.feather']

In [3]:
df=pd.read_feather(dp.path+'0.feather.feather')

In [4]:
df.head()

,eid,verbal_interview_duration_f3_0_0,biometrics_duration_f4_0_0,sample_collection_duration_f5_0_0,conclusion_duration_f6_0_0,heel_ultrasound_method_f19_0_0,weight_method_f21_0_0,spirometry_method_f23_0_0,sex_f31_0_0,year_of_birth_f34_0_0,...,astigmatism_angle_left_f5089_0_5,astigmatism_angle_left_f5089_0_6,astigmatism_angle_left_f5089_0_7,astigmatism_angle_left_f5089_0_8,astigmatism_angle_left_f5089_0_9,refractometry_result_unreliable_left_f5090_0_0,refractometry_result_unreliable_left_f5090_0_1,refractometry_result_unreliable_left_f5090_0_2,refractometry_result_unreliable_left_f5090_0_3,refractometry_result_unreliable_left_f5090_0_4
0,1000014,752.0,650.0,275.0,20.0,Manual entry,Direct entry,Direct entry,Female,1948,...,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan
1,1000023,400.0,551.0,250.0,14.0,Manual entry,Direct entry,Direct entry,Female,1966,...,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan
2,1000030,238.0,382.0,233.0,9.0,Direct entry,Direct entry,Direct entry,Female,1945,...,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan
3,1000041,741.0,366.0,103.0,39.0,Direct entry,Direct entry,Direct entry,Female,1943,...,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan
4,1000059,458.0,582.0,202.0,47.0,nan,Direct entry,Direct entry,Female,1957,...,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan


In [5]:
df_model=pd.read_parquet(dp.path+'df_all_final2022-04-13.parquet')

In [7]:
colsuse=[c for c in df.columns if c not in df_model.columns]

In [8]:
len(colsuse)

836

In [28]:
import re
pop_cols=[c for c in colsuse if df.loc[df[c]!='nan',c].count()>300000 and not re.search('device_id|snapbutton|time_blood',c)]

In [29]:
len(pop_cols)

85

In [30]:
import random

In [35]:
lkup=ds.field_name_full_file

In [36]:
v='forced_expiratory_volume_in_1second_fev1_f3063_0_2'
lkup.loc[lkup['col.name']==v,]

,Unnamed: 0,Unnamed: 0_x,field.showcase,field.html,field.tab,col.type,col.name,counts,Unnamed: 8,FieldID,...,Array,Coding,Notes,Link,field_type0,field_type1,field_type2,field_type3,Brain,Category.1
1540,1538.0,1539,3063,3063-0.2,f.3063.0.2,Continuous,forced_expiratory_volume_in_1second_fev1_f3063...,NaN,NaN,3063,...,3.0,NaN,FEV1 value calculated from blow,http://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=...,Assessment Centre,Physical measures,Spirometry,NaN,NaN,NaN


In [41]:
popcols_lkup=lkup.loc[lkup['col.name'].isin(pop_cols),]

In [69]:
orig_lkup['field_type2'].value_counts()

 Lifestyle and environment           68
 Genotyping process and sample QC    53
 Blood count                         31
 Blood count                         31
 Blood biochemistry                  26
 Reaction time                       25
 Psychosocial factors                22
 Health and medical history          17
 Residential air pollution           17
 Early life factors                   9
 Blood pressure                       8
 Pairs matching                       8
 Anthropometry                        6
 Residential noise pollution          5
 Family history                       4
 Summary Administration               4
 Reception                            3
 Sociodemographics                    3
 Record-level access                  3
 Spirometry                           2
 Operations                           2
 Medical conditions                   2
 Hand grip strength                   2
 Medications                          1
 Blood biochemistry                   1


In [76]:
mask=(lkup['col.name'].str.contains('worrier'))&pd.notnull(lkup['col.name'])
lkup.loc[mask,]

,Unnamed: 0,Unnamed: 0_x,field.showcase,field.html,field.tab,col.type,col.name,counts,Unnamed: 8,FieldID,...,Array,Coding,Notes,Link,field_type0,field_type1,field_type2,field_type3,Brain,Category.1
1167,1166.0,1167,1980,1980-0.0,f.1980.0.0,Categorical (single),worrier_anxious_feelings_f1980_0_0,9988.0,"{'Yes': 5383, 'No': 4316, 'Do not know': 269, ...",1980,...,1.0,100349.0,"ACE touchscreen question ""Are you a worrier?"" ...",http://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=...,Assessment Centre,Touchscreen,Psychosocial factors,Mental health,NaN,NaN
1168,1167.0,1168,1980,1980-1.0,f.1980.1.0,Categorical (single),worrier_anxious_feelings_f1980_1_0,NaN,NaN,1980,...,1.0,100349.0,"ACE touchscreen question ""Are you a worrier?"" ...",http://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=...,Assessment Centre,Touchscreen,Psychosocial factors,Mental health,NaN,NaN
1169,1168.0,1169,1980,1980-2.0,f.1980.2.0,Categorical (single),worrier_anxious_feelings_f1980_2_0,NaN,NaN,1980,...,1.0,100349.0,"ACE touchscreen question ""Are you a worrier?"" ...",http://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=...,Assessment Centre,Touchscreen,Psychosocial factors,Mental health,NaN,NaN
1170,1169.0,1170,1980,1980-3.0,f.1980.3.0,Categorical (single),worrier_anxious_feelings_f1980_3_0,NaN,NaN,1980,...,1.0,100349.0,"ACE touchscreen question ""Are you a worrier?"" ...",http://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=...,Assessment Centre,Touchscreen,Psychosocial factors,Mental health,NaN,NaN


In [79]:
[c for c in df_model.columns if len([y for y in list(lkup['col.name']) if re.search(y,c)])>0]

TypeError: first argument must be string or compiled pattern

In [42]:
popcols_lkup['field_type2'].value_counts()

 Spirometry                     26
 Lifestyle and environment      22
 Process durations               4
 Sociodemographics               4
 Bone-densitometry of heel       3
 Blood sample collection         3
 Reception                       3
 Anthropometry                   2
 Consent                         2
 Pairs matching                  2
 Early life factors              2
 Blood pressure                  2
 Medical conditions              1
 Operations                      1
 Family history                  1
 Health and medical history      1
Name: field_type2, dtype: int64

In [67]:
orig_lkup=lkup.loc[lkup['col.name'].isin(df_model.columns),]

In [65]:
def findvars(df=popcols_lkup,breakdown='Spirometry'):
    mask=(df['field_type2'].str.contains(breakdown,regex=True))&pd.notnull(df['field_type2'])
    return np.asarray(df.loc[mask,'Field'])
    

In [68]:
findvars(df=orig_lkup,breakdown='Sociodemographics|Lifes')

array(['Length of time at current address', 'Number in household',
       'Average total household income before tax',
       'Number of days/week walked 10+ minutes', 'Duration of walks',
       'Number of days/week of moderate physical activity 10+ minutes',
       'Duration of moderate activity',
       'Number of days/week of vigorous physical activity 10+ minutes',
       'Usual walking pace',
       'Frequency of stair climbing in last 4 weeks',
       'Time spend outdoors in summer', 'Time spent outdoors in winter',
       'Time spent watching television (TV)', 'Time spent using computer',
       'Time spent driving', 'Drive faster than motorway speed limit',
       'Length of mobile phone use',
       'Weekly usage of mobile phone in last 3 months',
       'Hands-free device/speakerphone use with mobile phone in last 3 month',
       'Difference in mobile phone use compared to two years previously',
       'Sleep duration', 'Getting up in morning', 'Nap during day',
       'Sle

In [66]:
findvars(breakdown='Sociodemographics|Lifes')

array(['Type of accommodation lived in',
       'Own or rent accommodation lived in',
       'Number of vehicles in household',
       'Age completed full time education',
       'Frequency of walking for pleasure in last 4 weeks',
       'Duration walking for pleasure',
       'Usual side of head for mobile phone use',
       'Morning/evening person (chronotype)', 'Milk type used',
       'Spread type', 'Bread type', 'Cereal type', 'Coffee type',
       'Major dietary changes in the last 5 years', 'Variation in diet',
       'Alcohol intake frequency.', 'Average weekly red wine intake',
       'Average weekly champagne plus white wine intake',
       'Average weekly beer plus cider intake',
       'Average weekly spirits intake',
       'Average weekly fortified wine intake',
       'Alcohol usually taken with meals', 'Skin colour',
       'Ease of skin tanning', 'Hair colour (natural, before greying)',
       'Answered sexual history questions'], dtype=object)

In [47]:
for c in popcols_lkup['field_type2'].unique():
    print(str(c)+':',np.asarray(popcols_lkup.loc[popcols_lkup['field_type2']==c,'Field']))
    

 Process durations: ['Verbal interview duration' 'Biometrics duration'
 'Sample collection duration' 'Conclusion duration']
 Bone-densitometry of heel: ['Heel ultrasound method' 'Foot measured for bone density'
 'Fractured heel']
 Anthropometry : ['Weight method' 'Seating box height']
 Spirometry: ['Spirometry method' 'Result ranking' 'Result ranking' 'Result ranking'
 'Time of blow measurement' 'Time of blow measurement'
 'Acceptability of each blow result' 'Acceptability of each blow result'
 'Acceptability of each blow result' 'Forced vital capacity (FVC)'
 'Forced expiratory volume in 1-second (FEV1)'
 'Forced expiratory volume in 1-second (FEV1)'
 'Forced expiratory volume in 1-second (FEV1)'
 'Peak expiratory flow (PEF)' 'Peak expiratory flow (PEF)'
 'Peak expiratory flow (PEF)' 'Ordering of blows' 'Ordering of blows'
 'Ordering of blows' 'Data points for blow' 'Data points for blow'
 'Contra-indications for spirometry' 'Caffeine drink within last hour'
 'Used an inhaler for ches

In [37]:
lkup['field_type2'].value_counts()

 Brain MRI                   4398
 ECG during exercise         1413
 Eye measures                1316
 Employment history          1251
 Hearing test                1244
                             ... 
 COPD outcomes                  2
 Asthma outcomes                2
 Saliva sample collection       1
 Urine sample collection        1
 HLA                            1
Name: field_type2, Length: 138, dtype: int64

In [31]:
[c for c in pop_cols if random.random()<0.2]

['spirometry_method_f23_0_0',
 'was_blood_sampling_attempted_f35_0_0',
 'date_of_consenting_to_join_uk_biobank_f200_0_0',
 'program_tactus_version_id_compiler_timestamp_f393_0_0',
 'usual_side_of_head_for_mobile_phone_use_f1150_0_0',
 'ease_of_skin_tanning_f1727_0_0',
 'fathers_age_at_death_f1807_0_0',
 'acceptability_of_each_blow_result_f3061_0_0',
 'acceptability_of_each_blow_result_f3061_0_2',
 'forced_expiratory_volume_in_1second_fev1_f3063_0_1',
 'forced_expiratory_volume_in_1second_fev1_f3063_0_2',
 'peak_expiratory_flow_pef_f3064_0_1',
 'ordering_of_blows_f3065_0_2',
 'caffeine_drink_within_last_hour_f3089_0_0']

In [27]:
df['caffeine_drink_within_last_hour_f3089_0_0'].value_counts()

No     449683
nan     42494
Yes     10234
Name: caffeine_drink_within_last_hour_f3089_0_0, dtype: int64

In [15]:
df[pop_cols].sample(20)

,verbal_interview_duration_f3_0_0,biometrics_duration_f4_0_0,sample_collection_duration_f5_0_0,conclusion_duration_f6_0_0,heel_ultrasound_method_f19_0_0,weight_method_f21_0_0,spirometry_method_f23_0_0,was_blood_sampling_attempted_f35_0_0,blood_pressure_device_id_f36_0_0,hand_grip_dynamometer_device_id_f38_0_0,...,number_of_measurements_made_f3137_0_0,time_blood_sample_collected_f3166_0_0,time_blood_sample_collected_f3166_0_1,time_blood_sample_collected_f3166_0_2,time_blood_sample_collected_f3166_0_3,time_blood_sample_collected_f3166_0_4,time_blood_sample_collected_f3166_0_5,hearing_aid_user_f3393_0_0,method_of_measuring_blood_pressure_f4081_0_0,method_of_measuring_blood_pressure_f4081_0_1
462524,1002.0,335.0,337.0,98.0,Manual entry,Direct entry,nan,Yes,1039.0,1354.0,...,NaN,2007-08-29 13:16:09,2007-08-29 13:16:11,2007-08-29 13:16:12,2007-08-29 13:16:14,2007-08-29 13:16:16,2007-08-29 13:16:18,nan,Direct entry,Direct entry
206775,219.0,577.0,206.0,10.0,Direct entry,Direct entry,Cannot be measured,Yes,1046.0,4565.0,...,NaN,2009-04-14 14:37:15,2009-04-14 14:37:21,2009-04-14 14:37:17,2009-04-14 14:37:12,2009-04-14 14:37:09,2009-04-14 14:37:21,nan,Direct entry,Direct entry
474877,628.0,391.0,362.0,27.0,Direct entry,Direct entry,Direct entry,Yes,3156.0,1348.0,...,3.0,2008-07-16 14:17:36,2008-07-16 14:17:40,2008-07-16 14:17:38,2008-07-16 14:17:42,2008-07-16 14:17:44,2008-07-16 14:17:45,No,Direct entry,Direct entry
45514,280.0,511.0,507.0,25.0,Direct entry,Direct entry,Direct entry,Yes,1058.0,3107.0,...,3.0,2008-08-11 09:59:58,2008-08-11 10:00:05,2008-08-11 10:00:08,2008-08-11 10:00:10,2008-08-11 10:00:13,2008-08-11 10:00:15,No,Direct entry,Direct entry
417873,267.0,536.0,160.0,15.0,nan,Direct entry,Direct entry,Yes,2818.0,6988.0,...,3.0,2010-07-03 17:28:16,2010-07-03 17:28:18,2010-07-03 17:28:21,2010-07-03 17:28:20,2010-07-03 17:28:24,2010-07-03 17:28:26,No,Direct entry,Direct entry
372311,351.0,409.0,259.0,9.0,Direct entry,Direct entry,Direct entry,Yes,1046.0,3745.0,...,3.0,2009-04-30 15:02:39,2009-04-30 15:02:38,2009-04-30 15:02:35,2009-04-30 15:02:27,2009-04-30 15:02:29,2009-04-30 15:02:32,nan,Direct entry,Direct entry
200673,680.0,560.0,251.0,13.0,nan,Direct entry,Direct entry,Yes,1053.0,7055.0,...,2.0,2009-10-28 10:24:13,2009-10-28 10:24:15,2009-10-28 10:24:10,2009-10-28 10:24:09,2009-10-28 10:24:17,2009-10-28 10:24:19,No,Direct entry,Direct entry
176707,307.0,442.0,185.0,7.0,Direct entry,Direct entry,Direct entry,Yes,1039.0,6534.0,...,2.0,2009-06-24 14:53:26,2009-06-24 14:53:28,2009-06-24 14:53:30,2009-06-24 14:53:33,2009-06-24 14:53:35,2009-06-24 14:53:38,nan,Direct entry,Direct entry
166202,309.0,530.0,188.0,13.0,nan,Direct entry,Direct entry,Yes,1046.0,6389.0,...,3.0,2009-10-29 16:36:20,2009-10-29 16:36:18,2009-10-29 16:36:24,2009-10-29 16:36:24,2009-10-29 16:36:27,2009-10-29 16:36:29,No,Direct entry,Direct entry
103044,400.0,691.0,258.0,33.0,Direct entry,Direct entry,Direct entry,Yes,2818.0,3836.0,...,3.0,2008-12-09 18:00:15,2008-12-09 18:00:17,2008-12-09 18:00:19,2008-12-09 18:00:21,2008-12-09 18:00:23,2008-12-09 18:00:24,nan,Direct entry,Direct entry
